# Import modelflow

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from ipywidgets import interact,Dropdown,Checkbox,Tab
from IPython.display import display, clear_output,Latex, Markdown
import pandas as pd

import modeljupyter as mj
from modelvis import waterplot


from modelclass import model
model.widescreen()

import modelwidget

# Load model and data, and run 

In [2]:
if not 'masia' in locals():
    print('Load model')
    masia,baseline = model.modelload('Asia.pcim',run=1,silent=1)
from inject import  inject
inject(masia)

Load model


In [3]:
xx = masia(baseline,stats=0,progressbar=1)

Solving Asia:   0%|          | 0/30

In [4]:
co2_input_var = [var for var in masia.totgraph.predecessors('WLD_CO2') if var.endswith('CO2')]

In [5]:
co2_baseline = baseline.loc[:,co2_input_var].pipe(lambda df:df.rename(columns={c : masia.iso_dict.get(cname := c.split('_')[0],cname) for c in df.columns}))

# Variables to inputwidget

In [6]:
outvar = 'GDI GCARBR CO2 LNN'.split()
displayvar = 'WLD_CO2 ASP_CO2 G20_CO2 '

#  masia.countries_GCARBR_A is list of all countries in the model with a countries_GCARBR_A 
#  masia.countries_ASEAN is list of all countries in ASEAN 
countries_rest = [country for country in masia.countries_GCARBR_A if country not in masia.countries_ASEAN]

slidedef_allasean =  {'All ASEAN countries' : {'var': ' '.join([f'{country}_GCARBR_A' for country in masia.countries_ASEAN ])
                          , 'value': 0.0,'min':0,'max':100,'op':'+impulse','step':10,'dec':0}}
slidedef_allrest =  {'Rest of the world' : {'var': 
    ' '.join([f'{country}_GCARBR_A' for country in countries_rest])
                          , 'value': 0.0,'min':0,'max':100,'op':'+impulse','step':10,'dec':0}}
slidedef_countries = {masia.iso_dict.get(country,country)+', Extra' : 
                      {'var': f'{country}_GCARBR_A', 'value': 0.0,'min':-40,'max':40,'op':'+impulse','step':10,'dec':5} 
                      for country in  masia.countries_ASEAN}



In [7]:

from modelwidget import slidewidget, update_tab,update_input
from IPython.display import display

In [8]:
tabwidget = update_tab({'Areas': {**slidedef_allasean,**slidedef_allrest},
                        'Country add on':slidedef_countries})

tabwidget.tabs

In [9]:
testdf = baseline.copy()
tabwidget.update_df(testdf,masia.current_per)
zz = masia(testdf)

Solving Asia:   0%|          | 0/30

In [10]:
masia.exodif().T

""


In [11]:
f'{2.2:>29,.1f}'

'                          2.2'

In [12]:
this = update_input(masia,{'Area  wide carbon tax': {**slidedef_allasean,**slidedef_allrest},
                        'Single country add on':slidedef_countries})

this.wtotal

In [13]:
updatevar = [f'{country}_GCARBR_A' for country in masia.countries_ASEAN ]
smalldf = baseline.loc[2022:2027, updatevar]
smalldf 

,BRN_GCARBR_A,KHM_GCARBR_A,IDN_GCARBR_A,LAO_GCARBR_A,MYS_GCARBR_A,MMR_GCARBR_A,PHL_GCARBR_A,SGP_GCARBR_A,THA_GCARBR_A,VNM_GCARBR_A
2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
xx = modelwidget.sheetwidget(smalldf,transpose=0,trans={c:c.split('_')[0] for c in updatevar })

xx.wsheet

[autoreload of modelwidget failed: Traceback (most recent call last):
  File "C:\Users\bruger\miniconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\bruger\miniconda3\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\Users\bruger\miniconda3\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\Users\bruger\miniconda3\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_code
  File "<frozen importlib._bootstrap_external>", line 846, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\modelflow2\modelflow\modelwidget.py", line 6

TypeError: __init__() got an unexpected keyword argument 'trans'

In [ ]:
modelwidget.sheetwidget?

In [ ]:
acopy = baseline.copy()
xnew = xx.update_df(acopy)

In [ ]:
xnew.loc[2022:2027,[f'{country}_GCARBR_A' for country in masia.countries_ASEAN ][:3]] 

In [ ]:
xx.df_new_var